In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input, Model, Sequential
from keras.layers import Embedding, SimpleRNN, Dense
import pandas as pd 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model ploting
from keras.utils import plot_model

# Label encoding 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler

In [ ]:
def get_train_test(url, split_percent=0.8):
    df = pd.read_csv(url, usecols=[1], engine='python')
    data = np.array(df.values.astype('float32'))
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data).flatten()
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]
    return train_data, test_data, data
 
sunspots_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
train_data, test_data, data = get_train_test(sunspots_url)

In [ ]:
len(train_data)

In [ ]:
len(test_data)

In [ ]:
def get_XY(dat, time_steps):
    # Indices of target array
    Y_ind = np.arange(time_steps, len(dat), time_steps)
    Y = dat[Y_ind]
    # Prepare X
    rows_x = len(Y)
    X = dat[range(time_steps*rows_x)]
    X = np.reshape(X, (rows_x, time_steps, 1))    
    return X, Y
 
time_steps = 6
trainX, trainY = get_XY(train_data, time_steps)
testX, testY = get_XY(test_data, time_steps)

In [ ]:
trainX.shape

In [ ]:
trainY.shape

In [ ]:
trainX[0]

In [ ]:
trainY[0]

In [ ]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(time_steps, 1)))
model.add(Dense(units=1, activation='tanh'))
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

In [ ]:
plot_model(model, show_shapes=True)

In [ ]:
model.fit(trainX, trainY, epochs=10, batch_size=16, validation_data=(testX, testY))

In [ ]:
w = model.weights

In [ ]:
w[0]

In [ ]:
w[1]

In [ ]:
w[2]

In [ ]:
w[3]

In [ ]:
w[4]